In [1]:
import torch
import torchvision as tv
import os
from transformer_flow import Model
import utils
import pathlib

os.environ["CUDA_VISIBLE_DEVICES"] = "4" # set GPU

utils.set_random_seed(100)
notebook_output_path = pathlib.Path('runs/notebook')

In [2]:
def rescale(x, lo, hi): # WuYi
    """Rescale a tensor to [lo,hi]."""
    assert (lo < hi), f"[rescale] lo={lo} must be smaller than hi={hi}"
    old_width = torch.max(x) - torch.min(x)
    old_center = torch.min(x) + (old_width / 2.)
    new_width = float(hi - lo)
    new_center = lo + (new_width / 2.)
    # shift everything back to zero:
    x = x - old_center
    # rescale to correct width:
    x = x * (new_width / old_width)
    # shift everything to the new center:
    x = x + new_center
    # return:
    return x

In [3]:
dataset = 'mnist'
num_classes = 10
img_size = 28
channel_size = 1

# we use a small model for fast demonstration, increase the model size for better results
patch_size = 4
channels = 128
blocks = 4
layers_per_block = 4
# try different noise levels to see its effect
noise_std = 0.1

batch_size = 256
lr = 2e-3
# increase epochs for better results
epochs = 100
sample_freq = 10

if torch.cuda.is_available():
    device = 'cuda' 
elif torch.backends.mps.is_available():
    device = 'mps' # if on mac
else:
    device = 'cpu' # if mps not available
print(f'using device {device}')

fixed_noise = torch.randn(num_classes * 10, (img_size // patch_size)**2, channel_size * patch_size ** 2, device=device)
fixed_y = torch.arange(num_classes, device=device).view(-1, 1).repeat(1, 10).flatten()

transform = tv.transforms.Compose([
    tv.transforms.Resize((img_size, img_size)),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0.5,), (0.5,)),
])
data = tv.datasets.MNIST('.', transform=transform, train=True, download=True)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)

model = Model(in_channels=channel_size, img_size=img_size, patch_size=patch_size, 
              channels=channels, num_blocks=blocks, layers_per_block=layers_per_block,
              num_classes=num_classes).to(device)

optimizer = torch.optim.AdamW(model.parameters(), betas=(0.9, 0.95), lr=lr, weight_decay=1e-4)
lr_schedule = utils.CosineLRSchedule(optimizer, len(data_loader), epochs * len(data_loader), 1e-6, lr)

model_name = f'original'
sample_dir = notebook_output_path / f'{dataset}_samples_{model_name}'
ckpt_file = notebook_output_path / f'{dataset}_model_{model_name}.pth'
sample_dir.mkdir(exist_ok=True, parents=True)

for epoch in range(epochs):
    losses = 0
    for x, y in data_loader:
        x = x.to(device)
        eps = noise_std * torch.randn_like(x)
        x = x + eps
        x = rescale(x, 0.0001, 0.9999)
        y = y.to(device)
        optimizer.zero_grad()
        z, outputs, logdets = model(x, y)
        loss = model.get_loss(z, logdets)
        loss.backward()
        optimizer.step()
        lr_schedule.step()
        losses += loss.item()

    print(f"epoch {epoch} lr {optimizer.param_groups[0]['lr']:.6f} loss {losses / len(data_loader):.4f}")
    print('layer norms', ' '.join([f'{z.pow(2).mean():.4f}' for z in outputs]))
    print(f'logdet: {logdets.mean():.4f}, prior p: {0.5 * z.pow(2).mean():.4f}')
    if (epoch + 1) % sample_freq == 0:
        with torch.no_grad():
            samples = model.reverse(fixed_noise, fixed_y)
        tv.utils.save_image(samples, sample_dir / f'samples_{epoch:03d}.png', normalize=False, nrow=10)
        latents = model.unpatchify(z[:100])
        tv.utils.save_image(latents, sample_dir / f'latent_{epoch:03d}.png', normalize=False, nrow=10)
        print(f'sampling complete. Sample mean: {samples.mean():.4f}, std: {samples.std():.4f}, max: {samples.max():.4f}, min: {samples.min():.4f}')
        print(f'latent mean: {latents.mean():.4f}, std: {latents.std():.4f}')
    print('\n')
torch.save(model.state_dict(), ckpt_file)

using device cuda
Number of parameters: 3.23M
epoch 0 lr 0.002000 loss -0.0959
layer norms 3.1774 2.8396 1.6584 0.8780
logdet: 0.8557, prior p: 0.4390


epoch 1 lr 0.001999 loss -0.4592
layer norms 3.1157 2.7451 1.6982 0.9084
logdet: 0.9933, prior p: 0.4542


epoch 2 lr 0.001998 loss -0.5306
layer norms 3.3822 3.0416 2.0015 1.1273
logdet: 1.0928, prior p: 0.5637


epoch 3 lr 0.001995 loss -0.5668
layer norms 3.2899 3.0536 1.9957 1.0206
logdet: 1.1065, prior p: 0.5103


epoch 4 lr 0.001992 loss -0.5813
layer norms 3.6279 3.5082 2.2640 0.9384
logdet: 1.0053, prior p: 0.4692


epoch 5 lr 0.001987 loss -0.5940
layer norms 3.1869 3.0346 2.0453 0.9632
logdet: 1.1404, prior p: 0.4816


epoch 6 lr 0.001982 loss -0.5990
layer norms 3.6849 3.7775 2.6632 0.9648
logdet: 1.0428, prior p: 0.4824


epoch 7 lr 0.001975 loss -0.6034
layer norms 3.6045 3.8020 2.5681 0.9280
logdet: 1.0465, prior p: 0.4640


epoch 8 lr 0.001968 loss -0.6057
layer norms 3.6082 3.8073 2.8070 0.9810
logdet: 1.1007, prior p: 

In [4]:
# now we can also evaluate the model by turning it into a classifier with Bayes rule, p(y|x) = p(y)p(x|y)/p(x)
data = tv.datasets.MNIST('.', transform=transform, train=False, download=False)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=False)
num_correct = 0
num_examples = 0
for x, y in data_loader:
    x = x.to(device)
    y = y.to(device)
    eps = noise_std * torch.randn_like(x)
    x = x.repeat(num_classes, 1, 1, 1)
    y_ = torch.arange(num_classes, device=device).view(-1, 1).repeat(1, y.size(0)).flatten()
    with torch.no_grad():
        z, outputs, logdets = model(x, y_)
        losses = 0.5 * z.pow(2).mean(dim=[1, 2]) - logdets # keep the batch dimension
        pred = losses.reshape(num_classes, y.size(0)).argmin(dim=0)
    num_correct += (pred == y).sum()
    num_examples += y.size(0)
print(f'Accuracy %{100 * num_correct / num_examples:.2f}')

Warning! NaN detected in patchify
Warning! NaN detected in block 0 logdet
Warning! NaN detected in block 0 output
Warning! NaN detected in block 1 logdet
Warning! NaN detected in block 1 output
Warning! NaN detected in block 2 logdet
Warning! NaN detected in block 2 output
Warning! NaN detected in block 3 logdet
Warning! NaN detected in block 3 output
Warning! NaN detected in patchify
Warning! NaN detected in block 0 logdet
Warning! NaN detected in block 0 output
Warning! NaN detected in block 1 logdet
Warning! NaN detected in block 1 output
Warning! NaN detected in block 2 logdet
Warning! NaN detected in block 2 output
Warning! NaN detected in block 3 logdet
Warning! NaN detected in block 3 output
Warning! NaN detected in patchify
Warning! NaN detected in block 0 logdet
Warning! NaN detected in block 0 output
Warning! NaN detected in block 1 logdet
Warning! NaN detected in block 1 output
Warning! NaN detected in block 2 logdet
Warning! NaN detected in block 2 output
Warning! NaN detec

In [5]:
# import torch
# import os
# from transformer_flow import Model
# import utils

# os.environ["CUDA_VISIBLE_DEVICES"] = "4" # set GPU
# utils.set_random_seed(100)

# num_classes = 10
# img_size = 28
# channel_size = 1

# # we use a small model for fast demonstration, increase the model size for better results
# patch_size = 4
# channels = 128
# blocks = 4
# layers_per_block = 4
# # try different noise levels to see its effect
# noise_std = 0.1

# if torch.cuda.is_available():
#     device = 'cuda' 
# elif torch.backends.mps.is_available():
#     device = 'mps' # if on mac
# else:
#     device = 'cpu' # if mps not available
# print(f'using device {device}')

# fixed_noise = torch.randn(num_classes * 10, (img_size // patch_size)**2, channel_size * patch_size ** 2, device=device)
# # fixed_noise = torch.randn(num_classes * 10, (img_size // patch_size)**2, channels, device=device)
# fixed_y = torch.arange(num_classes, device=device).view(-1, 1).repeat(1, 10).flatten()

# # load the model
# model = Model(in_channels=channel_size, img_size=img_size, patch_size=patch_size, 
#               channels=channels, num_blocks=blocks, layers_per_block=layers_per_block,
#               num_classes=num_classes).to(device)
# model.load_state_dict(torch.load("runs/notebook/mnist_model_4_128_4_4_0.10_linears_with_residual.pth"))
# model.eval()

# with torch.no_grad():
#     samples = model.reverse(fixed_noise, fixed_y)
#     # print the mean and std of the samples
#     mean = samples.mean(dim=[0, 2, 3])
#     std = samples.std(dim=[0, 2, 3])
#     print(f'mean: {mean}, std: {std}')